# Codigo

In [8]:
import os
import sqlite3
import pandas as pd

# Configuración de parámetros generales
directorio = 'C:/Users/Andri/Archivos'  # Cambia esta ruta a la carpeta correspondiente

# Función para configurar la base de datos
def configurar_base_datos():
    conn = sqlite3.connect('cooperativa.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS respaldo_flujo_efectivo (
        fecha TEXT,
        mesReporte INTEGER,
        añoReporte INTEGER,
        tipoRegistro TEXT,
        monto REAL
    )
    ''')
    conn.commit()
    print("Base de datos configurada y tabla creada/verificada.")
    return conn, cursor

# Función para encontrar el archivo de reporte más reciente
def obtener_archivo_mas_reciente(directorio):
    """Obtiene el archivo .csv o .txt más reciente del directorio especificado."""
    try:
        archivo = sorted([f for f in os.listdir(directorio) if f.endswith(('.csv', '.txt'))], reverse=True)[0]
        print(f"Archivo encontrado: {archivo}")
        return os.path.join(directorio, archivo)
    except IndexError:
        print("No se encontraron archivos .csv o .txt en el directorio especificado.")
        raise FileNotFoundError("No hay archivos de reporte en la carpeta.")

# Función para cargar datos y validar con retroalimentación detallada
def cargar_y_validar_datos(ruta_archivo):
    """Carga los datos y muestra los registros con errores específicos."""
    try:
        # Cargar archivo
        if ruta_archivo.endswith('.csv'):
            df = pd.read_csv(ruta_archivo)
        elif ruta_archivo.endswith('.txt'):
            df = pd.read_csv(ruta_archivo, sep='\t')
        print("Datos cargados exitosamente.")
        
        # Crear listas para identificar errores
        errores = {'campo': [], 'valor': [], 'descripcion': []}

        # Validación de cada registro
        for index, row in df.iterrows():
            if pd.isnull(row['fecha']):
                errores['campo'].append('fecha')
                errores['valor'].append(row['fecha'])
                errores['descripcion'].append('Fecha faltante')
            elif pd.to_datetime(row['fecha'], errors='coerce') is pd.NaT:
                errores['campo'].append('fecha')
                errores['valor'].append(row['fecha'])
                errores['descripcion'].append('Formato de fecha incorrecto')

            if pd.isnull(row['mesReporte']):
                errores['campo'].append('mesReporte')
                errores['valor'].append(row['mesReporte'])
                errores['descripcion'].append('Mes faltante')
            elif not isinstance(row['mesReporte'], int):
                errores['campo'].append('mesReporte')
                errores['valor'].append(row['mesReporte'])
                errores['descripcion'].append('Mes debe ser un número entero')

            if pd.isnull(row['añoReporte']):
                errores['campo'].append('añoReporte')
                errores['valor'].append(row['añoReporte'])
                errores['descripcion'].append('Año faltante')
            elif not isinstance(row['añoReporte'], int):
                errores['campo'].append('añoReporte')
                errores['valor'].append(row['añoReporte'])
                errores['descripcion'].append('Año debe ser un número entero')

            if pd.isnull(row['tipoRegistro']):
                errores['campo'].append('tipoRegistro')
                errores['valor'].append(row['tipoRegistro'])
                errores['descripcion'].append('Tipo de registro faltante')
            elif row['tipoRegistro'] not in ['Ingreso', 'Egreso']:
                errores['campo'].append('tipoRegistro')
                errores['valor'].append(row['tipoRegistro'])
                errores['descripcion'].append('Tipo de registro debe ser "Ingreso" o "Egreso"')

            if pd.isnull(row['monto']):
                errores['campo'].append('monto')
                errores['valor'].append(row['monto'])
                errores['descripcion'].append('Monto faltante')
            elif not isinstance(row['monto'], (int, float)):
                errores['campo'].append('monto')
                errores['valor'].append(row['monto'])
                errores['descripcion'].append('Monto debe ser un número')

        # Crear DataFrame de errores y mostrarlo
        errores_df = pd.DataFrame(errores)
        if not errores_df.empty:
            print("Registros con errores:")
            print(errores_df)
        else:
            print("Todos los registros cumplen con los criterios de validación.")

        # Filtrar y limpiar datos válidos
        df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce').dt.strftime('%Y-%m-%d')
        df.dropna(subset=['fecha', 'mesReporte', 'añoReporte', 'tipoRegistro', 'monto'], inplace=True)
        df = df[(df['tipoRegistro'].isin(['Ingreso', 'Egreso'])) & (df['monto'].apply(lambda x: isinstance(x, (int, float))))]
        df.drop_duplicates(subset=['fecha', 'tipoRegistro', 'monto'], inplace=True)
        
        return df

    except Exception as e:
        print(f"Error durante la carga y validación de datos: {e}")
        raise

# Función para insertar datos en la base de datos
def insertar_datos(cursor, df):
    """Inserta los datos validados en la tabla de respaldo en la base de datos."""
    registros_insertados = 0
    for index, row in df.iterrows():
        try:
            cursor.execute('''
            INSERT INTO respaldo_flujo_efectivo (fecha, mesReporte, añoReporte, tipoRegistro, monto) 
            VALUES (?, ?, ?, ?, ?)
            ''', (row['fecha'], row['mesReporte'], row['añoReporte'], row['tipoRegistro'], row['monto']))
            registros_insertados += 1
        except sqlite3.IntegrityError as e:
            print(f"Error de integridad al insertar el registro {index}: {e}")
    print(f"Total de registros insertados: {registros_insertados}")

# Función para verificar la cantidad de registros insertados
def verificar_insercion(cursor, df):
    """Verifica que la cantidad de registros insertados coincida con los datos procesados."""
    cursor.execute('SELECT COUNT(*) FROM respaldo_flujo_efectivo')
    total_registros = cursor.fetchone()[0]
    print(f"Total de registros en la base de datos: {total_registros}")

    total_registros_df = len(df)
    print(f"Total de registros procesados e insertados desde el archivo: {total_registros_df}")

    if total_registros_df == total_registros:
        print("Verificación exitosa: todos los registros fueron insertados correctamente.")
    else:
        print("Advertencia: Hubo una discrepancia en el número de registros.")

# Proceso principal
def main():
    print("Iniciando proceso de transferencia de datos...")

    # Configurar base de datos
    conn, cursor = configurar_base_datos()

    try:
        # Obtener el archivo más reciente y cargar datos
        ruta_archivo = obtener_archivo_mas_reciente(directorio)
        df = cargar_y_validar_datos(ruta_archivo)
        
        # Insertar datos en la base de datos
        insertar_datos(cursor, df)
        conn.commit()
        
        # Verificación final
        verificar_insercion(cursor, df)
        
        print("Proceso de transferencia de datos completado exitosamente.")
    except FileNotFoundError as fnf_error:
        print(fnf_error)
    except Exception as e:
        print(f"Se produjo un error inesperado: {e}")
    finally:
        conn.close()
        print("Conexión a la base de datos cerrada.")

# Ejecutar el proceso principal
if __name__ == "__main__":
    main()


Iniciando proceso de transferencia de datos...
Base de datos configurada y tabla creada/verificada.
Archivo encontrado: reporte_incorrecto.csv
Datos cargados exitosamente.
Registros con errores:
           campo         valor                                     descripcion
0     mesReporte            11                   Mes debe ser un número entero
1     añoReporte        2023.0                   Año debe ser un número entero
2          monto          4500                        Monto debe ser un número
3     mesReporte     noviembre                   Mes debe ser un número entero
4     añoReporte        2023.0                   Año debe ser un número entero
5   tipoRegistro        Salida  Tipo de registro debe ser "Ingreso" o "Egreso"
6          monto  two thousand                        Monto debe ser un número
7          fecha           NaN                                  Fecha faltante
8     mesReporte           NaN                                    Mes faltante
9     añoReport